The Bramble-Pasciak Transformation
===

In this lecture we derive the BP-Transfromation.

For an application of the BP-transformation see [application](useBP.ipynb).


In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw

from netgen.geom2d import SplineGeometry
geo = SplineGeometry()
geo.AddRectangle( (0, 0), (2, 0.41), bcs = ("wall", "outlet", "wall", "inlet"))
geo.AddCircle ( (0.2, 0.2), r=0.05, leftdomain=0, rightdomain=1, bc="cyl")
mesh = Mesh( geo.GenerateMesh(maxh=0.05)).Curve(3)

In [ ]:
V = VectorH1(mesh, order=2, dirichlet="wall|inlet|cyl")
V.SetOrder(TRIG,3)
V.Update()
Q = L2(mesh, order=1)

u,v = V.TnT()
p,q = Q.TnT()

bfa = BilinearForm(InnerProduct(grad(u),grad(v))*dx).Assemble()
bfb = BilinearForm(div(u)*q*dx).Assemble()

# prea = Preconditioner(bfa, "local")
prea = Preconditioner(bfa, "direct")
bfa.Assemble()

bfschur = BilinearForm(p*q*dx, diagonal=True).Assemble()
preschur = bfschur.mat.Inverse()

The saddle point system
$$
\left( \begin{array}{ccc} A & B^T \\ B & 0 \end{array} \right)
\left( \begin{array}{c} u \\ p  \end{array} \right)
=
\left( \begin{array}{c} f \\ g  \end{array} \right)
$$
is multiplied from the left by the transformation matrices
$$
\left( \begin{array}{ccc} A-\hat A & 0 \\ 0 & I \end{array} \right)
\left( \begin{array}{ccc} \hat I & 0 \\ B & -I \end{array} \right)
\left( \begin{array}{ccc} \hat A^{-1} & 0 \\ 0 & I \end{array} \right)
$$

to obtain the transformed system

$$
\left( \begin{array}{ccc} (A-\hat A) \hat A^{-1} A & (A-\hat A) \hat A^{-1} B^T \\ 
 B \hat A^{-1} (A - \hat A) & B \hat A^{-1} B^T \end{array} \right)
\left( \begin{array}{c} u \\ p  \end{array} \right)
=
\left( \begin{array}{c} (A-\hat A) A^{-1} f \\ B \hat A^{-1} f -g  \end{array} \right)
$$


In [ ]:
from ngsolve.la import EigenValues_Preconditioner
def BramblePasciakCG(A, B, f, g, preA, preS, maxit=1000, tol=1e-8):
    lam = EigenValues_Preconditioner(A,preA)
    print ("lammin/lammax = ", lam[0], '/', lam[-1])
    preA = 1.2/lam[0]*preA   # scaling
    
    x = BlockVector([f.CreateVector(), g.CreateVector()])
    w,r,p,ap = [x.CreateVector() for i in range(4)]
    
    # r.data = b
    # p.data = pre*r
    pru = (preA * f).Evaluate()
    r[0].data = A*pru - f
    r[1].data = B*pru - g
    p[0].data = pru    
    p[1].data = preS*r[1]
    
    wrn = InnerProduct(r,p)
    err0 = sqrt(wrn)
        
    x[:] = 0
    for it in range(maxit):
        # ap.data = A * p
        hv = (A * p[0] + B.T * p[1]).Evaluate()
        papu = (preA * hv).Evaluate()
        ap[0].data = A * papu - hv
        ap[1].data = B * (papu - p[0])
        
        pap = InnerProduct(p, ap)
        wr = wrn
        alpha = wr / pap
        
        x += alpha * p
        r -= alpha * ap
        pru -= alpha * papu

        # w.data = pre*r
        w[0].data = pru
        w[1].data = preS * r[1]
        
        wrn = InnerProduct(w, r)
        err = sqrt(wrn)
        print ("Iteration",it,"err=",err)
        if err < tol * err0: break
            
        beta = wrn / wr
        
        p *= beta
        p.data += w 
    
    return x[0], x[1]

In [ ]:
gfu = GridFunction(V)
gfp = GridFunction(Q)

uin = (1.5*4*y*(0.41-y)/(0.41*0.41), 0)
gfu.Set(uin, definedon=mesh.Boundaries("inlet"))

resf = (-bfa.mat * gfu.vec).Evaluate()
resg = (-bfb.mat * gfu.vec).Evaluate()

sol = BramblePasciakCG (A=bfa.mat, B=bfb.mat, f=resf, g=resg, preA=prea, preS=preschur)

gfu.vec.data += sol[0]
gfp.vec.data += sol[1]

Draw (gfu)
Draw (gfp);

for convenience of the reader, we started from this version of the usual preconditioned cg-iteration:

In [ ]:
def CG(A,b,pre, maxit=200, tol=1e-8):
    x = b.CreateVector()
    w = b.CreateVector()
    r = b.CreateVector()
    p = b.CreateVector()
    ap = b.CreateVector()

    r.data = b
    p.data = pre*r
    wrn = InnerProduct(r,p)
    err0 = sqrt(wrn)
    x[:] = 0
    
    for it in range(maxit):
        ap.data = A * p

        pap = InnerProduct(p, ap)
        wr = wrn
        alpha = wr / pap
        
        x.data += alpha * p
        r.data -= alpha * ap
        w.data = pre*r
        
        wrn = InnerProduct(w, r)
        err = sqrt(wrn)
        print ("Iteration",it,"err=",err)
        if err < tol * err0: break
            
        beta = wrn / wr
        
        p *= beta
        p.data += w 
        
    return x